In [2]:
from mnist import MNIST
import numpy as np
from scipy.special import expit
import time

In [3]:
# Step 1: Load image and label data
def load_mnist_data():

    # Initialize the MNIST object
    mndata = MNIST('')
    mndata.gz = True
    
    # Load training images and labels
    images, labels = mndata.load_training()
    
    # Load test images and labels
    test_images, test_labels = mndata.load_testing()

    return np.array(images), np.array(labels), np.array(test_images), np.array(test_labels)


In [4]:
def initialize_weights(hidden_size):
    weights_hidden = np.random.uniform(-0.01, 0.01, size=(hidden_size, 784))
    weights_output = np.random.uniform(-0.01, 0.01, size=(10, hidden_size))
    
    return weights_hidden, weights_output

In [121]:
def tanh_activation(x):
    return np.tanh(x)

In [120]:
def relu_activation(x):
    return np.maximum(x, 0)

In [119]:
def sigmoid_activation(x):
    y = expit(x)
    return y

In [8]:
    def desired_output_tanh(label,notDesiredValue):
        if notDesiredValue == -1:
            d = -np.ones((10,1))
            d[label] = 1
            return d
        else:
            d = np.zeros((10,1))
            d[label] = 1
            return d

In [9]:
def desired_output_sigmoid(label):
    desired = np.eye(10)[label]

    return desired

In [10]:
def find_error(output,desired):
    error = desired - output
    return error

In [12]:
def activation(x, ActivationFunction):
        return ActivationFunction(x)

In [13]:
def forward_pass(inputs, weights_input_hidden, weights_hidden_output, activation_hidden, activation_output):
    hidden_inputs = np.dot(weights_input_hidden, inputs)
    hidden_outputs = activation(hidden_inputs, activation_hidden)
    final_inputs = np.dot(weights_hidden_output, hidden_outputs)
    final_outputs = activation(final_inputs, activation_output)
    return hidden_outputs, final_outputs


In [44]:
def output_gradient(error,final_output):
    derivative=np.eye(10)*(1/2)*(1-final_output*final_output)
    gradient=np.dot(derivative,error)
    output_gradient=np.reshape(gradient,(10,1))
    return output_gradient

In [123]:
def output_gradient2(error,final_output):
    derivative=np.eye(10)*(final_output-final_output*final_output)
    gradient=np.dot(derivative,error)
    output_gradient=np.reshape(gradient,(10,1))
    return output_gradient

In [15]:
def input_gradient(layer_num,first_output,output_gradient,weight_output):
    derivative=np.eye(layer_num)*(1/2)*(1-first_output*first_output)
    gradient=np.dot(np.dot(derivative,np.transpose(weight_output)),output_gradient)
    input_gradient=np.reshape(gradient,(layer_num,1))
    return input_gradient

In [160]:
def relu_derivative(x):
    x[x>=0]=1
    x[x<0]=0
    return x

In [169]:
def input_gradient2(layer_num,first_output,output_gradient,weight_output):
    derivative=np.eye(layer_num)*relu_derivative(first_output)
    gradient=np.dot(np.dot(derivative,np.transpose(weight_output)),output_gradient)
    input_gradient=np.reshape(gradient,(layer_num,1))
    return input_gradient

In [99]:
def backpropagation_tanh(inputs, labels, weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num):
    hidden_outputs, final_outputs = forward_pass(inputs, weights_input_hidden, weights_hidden_output, activation_hidden, activation_output)
    desired = desired_output_tanh(labels, -1)
    error = find_error(final_outputs, desired)
    output_gradients = output_gradient2(error, final_outputs)
    weights_hidden_output_updated = weights_hidden_output + learning_rate * np.dot(output_gradients, hidden_outputs.T)
    input_gradients = input_gradient(layer_num, hidden_outputs, output_gradients, weights_hidden_output)
    weights_input_hidden_updated = weights_input_hidden + learning_rate * np.dot(input_gradients, inputs.T)
    
    return error, weights_hidden_output_updated, weights_input_hidden_updated


In [193]:
def backpropagation_tanh2(inputs, labels, weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num,lambda_coef):
    hidden_outputs, final_outputs = forward_pass(inputs, weights_input_hidden, weights_hidden_output, activation_hidden, activation_output)
    desired = desired_output_tanh(labels, -1)
    error = find_error(final_outputs, desired)
    output_gradients = output_gradient2(error, final_outputs)
    weights_hidden_output_updated = (1-learning_rate*lambda_coef)*weights_hidden_output + learning_rate * np.dot(output_gradients, hidden_outputs.T)
    input_gradients = input_gradient(layer_num, hidden_outputs, output_gradients, weights_hidden_output)
    weights_input_hidden_updated = (1-learning_rate*lambda_coef)*weights_input_hidden + learning_rate * np.dot(input_gradients, inputs.T)
    
    return error, weights_hidden_output_updated, weights_input_hidden_updated

In [194]:
def backpropagation_sigmoid(inputs, labels, weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num):
    hidden_outputs, final_outputs = forward_pass(inputs, weights_input_hidden, weights_hidden_output, activation_hidden, activation_output)
    desired = desired_output_tanh(labels, 0)
    error = find_error(final_outputs, desired)
    output_gradients = output_gradient2(error, final_outputs)
    weights_hidden_output_updated = weights_hidden_output + learning_rate * np.dot(output_gradients, hidden_outputs.T)
    input_gradients = input_gradient2(layer_num, hidden_outputs, output_gradients, weights_hidden_output)
    weights_input_hidden_updated = weights_input_hidden + learning_rate * np.dot(input_gradients, inputs.T)
    
    return error, weights_hidden_output_updated, weights_input_hidden_updated


In [198]:
def backpropagation_sigmoid2(inputs, labels, weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num,lambda_coef):
    hidden_outputs, final_outputs = forward_pass(inputs, weights_input_hidden, weights_hidden_output, activation_hidden, activation_output)
    desired = desired_output_tanh(labels, 0)
    error = find_error(final_outputs, desired)
    output_gradients = output_gradient2(error, final_outputs)
    weights_hidden_output_updated = (1-learning_rate*lambda_coef)*weights_hidden_output + learning_rate * np.dot(output_gradients,hidden_outputs.T)
    input_gradients = input_gradient2(layer_num, hidden_outputs, output_gradients, weights_hidden_output)
    weights_input_hidden_updated = (1-learning_rate*lambda_coef)*weights_input_hidden + learning_rate * np.dot(input_gradients, inputs.T)
    
    return error, weights_hidden_output_updated, weights_input_hidden_updated


In [103]:
def train(inputs, labels,test_images ,test_label,weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num, batch_size, epochs):     
    trainingError = 0.0
    startTime = time.time()
    
    for epoch in range(epochs):
        cumulativeError = 0.0
        
        for i in range(batch_size):
            flattened_data = np.reshape(inputs[i], (784, 1)) / 255
            error, weights_hidden_output, weights_input_hidden = backpropagation_tanh(flattened_data, labels[i], weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num)
            cumulativeError += np.sum(error**2 * 0.5)
        
        epochErrorMean = cumulativeError / batch_size
        trainingError += epochErrorMean
        
        print("Epoch Completed: ", epoch + 1, "\nMean Squares Error of Epoch: ", epochErrorMean)
    
    trainingErrorMean = trainingError / epochs
    stopTime = time.time()
    print("Training Completed!\nCPU Time: ", stopTime - startTime, " seconds", "\nOverall Error:", trainingErrorMean)
    
    cumulativeError_2 = 0.0
    numOfMissClassification = 0
    
    for i in range(10000):
        testData = np.reshape(test_images[i], (784, 1)) / 255
        layer_output,final_output=forward_pass(testData, weights_input_hidden, weights_hidden_output,activation_hidden, activation_output)
        desired=desired_output_tanh(test_label[i],-1) 
        error_2=find_error(final_output,desired)
        cumulativeError_2 += np.sum(error_2**2 * 0.5)
        
        predicted = np.argmax(final_output)
        
        if test_label[i] != predicted:
            numOfMissClassification += 1
    
    misclasificationPercentage = numOfMissClassification / 10000 * 100
            
    testSetErrorMean = cumulativeError_2 / 10000
    
    print("Number Of Misclasification:",numOfMissClassification,"\nError Percentage: %",misclasificationPercentage,"\nTest Data Mean Square Error:",testSetErrorMean)

# İlk deneme batch 1250 epoch 50 lr=0.01 N=300 tanh

In [47]:
Train_input,Train_label,Test_input,Test_label= load_mnist_data()


In [48]:
weights_hidden, weights_output=initialize_weights(300)

In [108]:
activation_hidden_1= tanh_activation
activation_output_1= tanh_activation

In [50]:
learning_rate=0.01
epochs=50
batch_size=1250
layer_num=300

In [51]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.859319419728675
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2703111760269532
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.8861540536364227
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.703623320513816
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5772910217783453
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.47785931249806063
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.4176432464859711
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.37650261764109316
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.34544523714273906
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.32015605383969714
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.2984026984290862
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.27896578830369123
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.2611276467353776
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.24475495646713935
Epoch Compl

# batch 1250 epoch 50 lr=0.05 N=300 tanh

In [387]:
weights_hidden, weights_output=initialize_weights(300)

In [24]:
activation_hidden_1= tanh_activation
activation_output_1= tanh_activation

In [389]:
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=300

In [390]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.2520932702594683
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.6824458972229046
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.516403195024465
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.449225882026016
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.38350991935790835
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.34132521039514274
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.31056251111144095
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.2733857731464881
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.2633568125898517
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.23787789670114318
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.21000127911568783
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.18943679039753705
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.183684297620952
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.1741921444018991
Epoch Comple

# batch 1250 epoch 50 lr=0.09 N=300 tanh

In [19]:
weights_hidden, weights_output=initialize_weights(300)

In [20]:
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=300

In [25]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.251274436116036
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.6802422783241082
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.5092366199880346
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.44412812696310966
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.38056218486976434
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.3496687964022909
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.3104222102835941
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.2768762787975626
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.23774189879496904
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.2243190674599141
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.2202547490376011
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.18804379523412715
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.18216403266387612
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.17503792014765623
Epoch Comp

# batch 1250 epoch 50 lr=0.01 N=500 tanh

In [26]:
weights_hidden, weights_output=initialize_weights(500)
learning_rate=0.01
epochs=50
batch_size=1250
layer_num=500

In [27]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.7863910485739882
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.1432341311457055
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.8059417580493782
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.649241688711455
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.5268379057973046
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4510541949952357
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.4009970922507539
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3645837429460448
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.33641299405652236
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.3132280500948612
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.29315630746562266
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2751302703726596
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.25852813219777465
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.24309045683245073
Epoch Comple

# batch 1250 epoch 50 lr=0.05 N=500 tanh

In [28]:
weights_hidden, weights_output=initialize_weights(500)
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=500

In [29]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.2160353097170993
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.6923413325840234
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.5505318829825554
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.5025221629196731
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.46078059445667685
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.39561196664822496
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.3940856376349105
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.34957347113040665
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.3140866160365159
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.2975645999432926
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.28137221193675777
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.26196283042267476
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.2431511422125047
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.2272530453938543
Epoch Comp

# batch 1250 epoch 50 lr=0.09 N=500 tanh

In [30]:
weights_hidden, weights_output=initialize_weights(500)
learning_rate=0.09
epochs=50
batch_size=1250
layer_num=500

In [31]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.3153655798247557
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.0432683044175295
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.8123150285139864
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.7897888606984101
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.7118801193671452
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.6620412500840557
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.5713727679346565
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.6177938694238444
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.5291580673034825
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.49108239278994387
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.48789444512049324
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.4871647999360091
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.4161941661463473
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.44541407142112777
Epoch Comple

# batch 1250 epoch 50 lr=0.01 N=1000 tanh

In [32]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.01
epochs=50
batch_size=1250
layer_num=1000

In [33]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.661825464727839
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.9803955937125222
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.7186678536868494
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.5707427797988545
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.4853344350735552
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4306205692193031
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.38986940092414957
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3569286303699408
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.33037554930804486
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.30848676965273286
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.2894061624799908
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.2723759237118888
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.25685959013279264
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.24241853467657087
Epoch Compl

# batch 1250 epoch 50 lr=0.05 N=1000 tanh

In [34]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=1000

In [35]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.1481759030722989
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.6827881974283255
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.5744295567280577
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.5189464953445021
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.4845869361442486
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.43426372788189765
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.42281969320003354
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.3914461247398152
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.3595423266567772
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.3440266738300212
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.3785098558873438
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.3466656256805334
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.3192733156657591
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.3076161235691417
Epoch Complet

# batch 1250 epoch 50 lr=0.09 N=1000 tanh

In [36]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.09
epochs=50
batch_size=1250
layer_num=1000

In [37]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.3613021217806964
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.1683135595182714
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.991636274493358
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.9040532995809976
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.8987350538852866
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.9485455880029147
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.849178897690414
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.9261175424049356
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.8831229531455967
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.7563079413762558
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.7997965975523702
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.7099608333690247
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.7548479589150078
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.7249604037736705
Epoch Completed: 

# batch 60000 epoch 50 lr=0.01 N=300 tanh

In [38]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.01
epochs=50
batch_size=60000
layer_num=300

In [39]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.42654074322600866
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.24644154185986855
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.18842121487664315
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.15173305082996053
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.12817467253336803
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.11121401655954707
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.09798971572510534
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.08762145267057418
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.07906872345613326
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.07195748959829769
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.06558610054952238
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.060204700869409766
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.05537144619868835
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.05106049861685127


# batch 60000 epoch 50 lr=0.05 N=300 tanh

In [40]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.05
epochs=50
batch_size=60000
layer_num=300

In [41]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.40639651983858227
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.25150859192060254
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.20679311779477325
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.18322389837653305
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.16821844303176955
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.15063555803337375
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.1432291269068177
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.13328733128750042
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.12490329639859327
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.11768625401066915
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.11364848365199305
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.11550801993699894
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.10720472497762652
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.10158929237142306
Ep

# batch 60000 epoch 50 lr=0.09 N=300 tanh

In [42]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.09
epochs=50
batch_size=60000
layer_num=300

In [43]:
train(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5304372629586815
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.3453965871096094
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.3043275407091378
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.2816227037302678
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.26778228664555837
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.25247570741576036
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.24370767681058317
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.23471029885773198
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.22995950842033983
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.22355729262636137
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.2213368466602324
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.22140819146812424
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.2220770377763252
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.20146247613713586
Epoch C

In [202]:
def train2(inputs, labels,test_images ,test_label,weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num, batch_size, epochs):     
    trainingError = 0.0
    startTime = time.time()
    
    for epoch in range(epochs):
        cumulativeError = 0.0
        
        for i in range(batch_size):
            flattened_data = np.reshape(inputs[i], (784, 1)) / 255
            error, weights_hidden_output, weights_input_hidden = backpropagation_sigmoid(flattened_data, labels[i], weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num)
            cumulativeError += np.sum(error**2 * 0.5)
        
        epochErrorMean = cumulativeError / batch_size
        trainingError += epochErrorMean
        
        print("Epoch Completed: ", epoch + 1, "\nMean Squares Error of Epoch: ", epochErrorMean)
    
    trainingErrorMean = trainingError / epochs
    stopTime = time.time()
    print("Training Completed!\nCPU Time: ", stopTime - startTime, " seconds", "\nOverall Error:", trainingErrorMean)
    
    cumulativeError_2 = 0.0
    numOfMissClassification = 0
    
    for i in range(10000):
        testData = np.reshape(test_images[i], (784, 1)) / 255
        layer_output,final_output=forward_pass(testData, weights_input_hidden, weights_hidden_output,activation_hidden, activation_output)
        desired=desired_output_tanh(test_label[i],0) 
        error_2=find_error(final_output,desired)
        cumulativeError_2 += np.sum(error_2**2 * 0.5)
        
        predicted = np.argmax(final_output)
        
        if test_label[i] != predicted:
            numOfMissClassification += 1
    
    misclasificationPercentage = numOfMissClassification / 10000 * 100
            
    testSetErrorMean = cumulativeError_2 / 10000
    
    print("Number Of Misclasification:",numOfMissClassification,"\nError Percentage: %",misclasificationPercentage,"\nTest Data Mean Square Error:",testSetErrorMean)

# Sigmoid lr=0.01 N= 300 Batch=1250


In [146]:
weights_hidden, weights_output=initialize_weights(300)

In [147]:
activation_hidden_1= relu_activation
activation_output_1= sigmoid_activation

In [174]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.01
epochs=50
batch_size=1250
layer_num=300

In [175]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5490966608460328
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4563014681172558
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.43955580122536303
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.4032789322171469
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.3564973306723562
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.314333508668047
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.2786635054337061
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.24331616175466939
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.2119473481379894
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.1895730804713971
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.1732315912929209
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.15941863304970186
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.14595160994676895
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.1328619253743287
Epoch Comple

# Sigmoid lr=0.05 N= 300 Batch=1250

In [176]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=300

In [177]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.44999268019595795
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.2723110922945972
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.16759125126198293
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.11866212715351478
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.09555631509714214
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.08204010587655773
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.07217137453551149
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.06421968799817959
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.05754569674661558
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.05182262789617115
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.046851216847284924
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.042291695593029005
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.03833899358594178
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.0349050775320725
E

# Sigmoid lr=0.09 N= 300 Batch=1250

In [178]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.09
epochs=50
batch_size=1250
layer_num=300

In [179]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.38999289138415805
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.1714244611048904
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.11125869162459087
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.08870023171833269
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.07446118151540489
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.0632526382768378
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.05501039239031205
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.0487440960064039
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.0431308963704836
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.03824657427268556
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.03512693081657597
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.030253307257076437
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.02804716435530987
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.026902152580568668
Epo

# Sigmoid lr=0.01 N= 500 Batch=1250

In [180]:
weights_hidden, weights_output=initialize_weights(500)
learning_rate=0.01
epochs=50
batch_size=1250
layer_num=500

In [181]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.542669937240716
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.45304692726569995
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.42801155456994366
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.3831285356124365
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.3327788856874811
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.2945486973880943
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.2580399034304393
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.22299708103677585
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.19689389333040838
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.1784653581448537
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.16326837508956202
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.14864758783575677
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.13473556113392463
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.12314386588207869
Epoch Co

# Sigmoid lr=0.05 N= 500 Batch=1250

In [182]:
weights_hidden, weights_output=initialize_weights(500)
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=500

In [183]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.4384125967702257
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.25047156776303836
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.15378011578286532
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.11231127179214966
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.09265060126692203
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.08028138530218634
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.07074716008787459
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.06322170900711784
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.05687138694717419
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.051130553332161495
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.04605459844145554
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.041653904000240924
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.03802200799989703
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.034928375852616246

# Sigmoid lr=0.09 N= 500 Batch=1250

In [184]:
weights_hidden, weights_output=initialize_weights(500)
learning_rate=0.09
epochs=50
batch_size=1250
layer_num=500

In [185]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.3742524733969621
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.16321022504202004
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.10956118500629895
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.0877484180610404
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.07442718741091535
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.06333189470890215
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.05518377962964794
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.047526977653276924
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.04387429093062984
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.039321904061657906
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.033683126514023776
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.03078105803526151
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.027738009621785988
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.02558683353704007

# Sigmoid lr=0.01 N= 1000 Batch=1250

In [186]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.01
epochs=50
batch_size=1250
layer_num=1000

In [187]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5295144732495662
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4453091618568404
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.4041963224355678
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.34523708321869356
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.2966913384659356
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.2532632832655168
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.2168311713584313
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.1917839715162773
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.17321514659163656
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.1565938875490386
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.14102946367297503
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.12820943832435244
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.11817534622917415
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.11009016820406974
Epoch Com

# Sigmoid lr=0.05 N= 1000 Batch=1250

In [188]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.05
epochs=50
batch_size=1250
layer_num=1000

In [189]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.41634875495706747
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.21947615669274678
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.13802303245256264
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.10556570796658084
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.08891343364727977
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.07761218898823999
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.0688222474385757
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.06146204694022738
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.05502618097297832
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.049416427332794624
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.04455391995146691
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.04039379657280112
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.03685387288428708
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.033752911613884995


# Sigmoid lr=0.09 N= 1000 Batch=1250

In [190]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.09
epochs=50
batch_size=1250
layer_num=1000

In [191]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.3429455245006791
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.14977765598811485
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.1049847398302768
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.08580839746130223
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.07321670352858706
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.06194352286105461
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.05417361137191133
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.04864920709375259
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.040797726866787946
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.036525500204666564
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.03400803990097158
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.03007345884422637
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.027086965097960655
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.025242846590544717

# son kısım

In [200]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.01
epochs=50
batch_size=10
layer_num=1000

In [203]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.247215125078162
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.2331712453584178
Epoch Completed:  3 
Mean Squares Error of Epoch:  1.2173965958891089
Epoch Completed:  4 
Mean Squares Error of Epoch:  1.1980725248104693
Epoch Completed:  5 
Mean Squares Error of Epoch:  1.1731701567316983
Epoch Completed:  6 
Mean Squares Error of Epoch:  1.1406729622196918
Epoch Completed:  7 
Mean Squares Error of Epoch:  1.0985022771334134
Epoch Completed:  8 
Mean Squares Error of Epoch:  1.0452876729956038
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.9811570689311649
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.9085174766399724
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.8322437228125897
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.7584752425116874
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.6924334104744116
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.6368881191780924
Epoch Completed:

In [204]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.01
epochs=50
batch_size=50
layer_num=1000

In [205]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.20803273944885
Epoch Completed:  2 
Mean Squares Error of Epoch:  1.0184911307684335
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.7050198076519313
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.5429953447655751
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.4937188432804435
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4758744614565329
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.4676654695381787
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.46315454621173496
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.46032240363917365
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.45834940839840416
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.45685657968491306
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.4556508170618909
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.45462441604685727
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.4537136429133842
Epoch Comple

In [206]:
weights_hidden, weights_output=initialize_weights(1000)
learning_rate=0.01
epochs=50
batch_size=100
layer_num=1000

In [207]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.1250942179654375
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.6329469979780286
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.48467785488632537
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.4649464661657643
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.4589884777500348
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4559902142731478
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.4539238739521521
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.4521934127761779
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.4505612030302506
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.4489039253019365
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.44713868774282484
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.4451950591735715
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.4430020287638054
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.44048152014082187
Epoch Comple

In [209]:
def train3(inputs, labels,test_images ,test_label,weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num, batch_size, epochs,lambda_coef):     
    trainingError = 0.0
    startTime = time.time()
    
    for epoch in range(epochs):
        cumulativeError = 0.0
        
        for i in range(batch_size):
            flattened_data = np.reshape(inputs[i], (784, 1)) / 255
            error, weights_hidden_output, weights_input_hidden = backpropagation_sigmoid2(flattened_data, labels[i], weights_hidden_output, weights_input_hidden, activation_hidden, activation_output, learning_rate, layer_num,lambda_coef)
            cumulativeError += np.sum(error**2 * 0.5)
        
        epochErrorMean = cumulativeError / batch_size
        trainingError += epochErrorMean
        
        print("Epoch Completed: ", epoch + 1, "\nMean Squares Error of Epoch: ", epochErrorMean)
    
    trainingErrorMean = trainingError / epochs
    stopTime = time.time()
    print("Training Completed!\nCPU Time: ", stopTime - startTime, " seconds", "\nOverall Error:", trainingErrorMean)
    
    cumulativeError_2 = 0.0
    numOfMissClassification = 0
    
    for i in range(10000):
        testData = np.reshape(test_images[i], (784, 1)) / 255
        layer_output,final_output=forward_pass(testData, weights_input_hidden, weights_hidden_output,activation_hidden, activation_output)
        desired=desired_output_tanh(test_label[i],0) 
        error_2=find_error(final_output,desired)
        cumulativeError_2 += np.sum(error_2**2 * 0.5)
        
        predicted = np.argmax(final_output)
        
        if test_label[i] != predicted:
            numOfMissClassification += 1
    
    misclasificationPercentage = numOfMissClassification / 10000 * 100
            
    testSetErrorMean = cumulativeError_2 / 10000
    
    print("Number Of Misclasification:",numOfMissClassification,"\nError Percentage: %",misclasificationPercentage,"\nTest Data Mean Square Error:",testSetErrorMean)

In [210]:
weights_hidden, weights_output=initialize_weights(300)
activation_hidden_1= relu_activation
activation_output_1= sigmoid_activation
batch_size=1250
learning_rate=0.01
lambda_coef=0.001
epochs=50
layer_num=300

In [213]:
train3(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs,lambda_coef)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.5552223401319203
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4569634516270435
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.44358639876761297
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.4085226924042471
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.35948328760860215
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.3173469102028302
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.28376944473324134
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.2513551688681435
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.22056096439929074
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.19704189619777193
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.18036936450680688
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.16736012129720712
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.15576343980095742
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.14394122443470939
Epoch 

In [218]:
weights_hidden, weights_output=initialize_weights(1000)
activation_hidden_1= relu_activation
activation_output_1= sigmoid_activation
batch_size=1250
learning_rate=0.01
lambda_coef=0.001
epochs=50
layer_num=1000

In [219]:
train3(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs,lambda_coef)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.526420635643888
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.4458028177221959
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.40772429317499564
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.35154066295290826
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.3035822230536042
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.2614487254649878
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.22447676601357203
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.1985133908797089
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.18028403925327682
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.16534882737076395
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.15110170190326272
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.13777627171859938
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.12689706400057574
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.11834623584583974
Epoch C

In [220]:
weights_hidden, weights_output=initialize_weights(300)
learning_rate=0.01
epochs=50
batch_size=60000
layer_num=300

In [221]:
train2(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs)

Epoch Completed:  1 
Mean Squares Error of Epoch:  0.15162465021934585
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.07361014565124696
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.06697500994471361
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.06350676690019828
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.06125548691459688
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.05958545619425713
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.058247125482473856
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.05714036809130575
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.05621285000898307
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.05541436194486249
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.054707533743202065
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.05406484757380091
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.05347947574920769
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.0529526087102312


In [227]:
weights_hidden, weights_output=initialize_weights(1000)
activation_hidden_1= relu_activation
activation_output_1= sigmoid_activation
batch_size=100
learning_rate=0.01
lambda_coef=0.001
epochs=50
layer_num=1000

In [228]:
train3(Train_input,Train_label,Test_input ,Test_label,weights_output, weights_hidden,activation_hidden_1, activation_output_1, learning_rate,layer_num,batch_size,epochs,lambda_coef)

Epoch Completed:  1 
Mean Squares Error of Epoch:  1.1264037194188028
Epoch Completed:  2 
Mean Squares Error of Epoch:  0.6353695950621537
Epoch Completed:  3 
Mean Squares Error of Epoch:  0.4855093981633077
Epoch Completed:  4 
Mean Squares Error of Epoch:  0.4655529870788492
Epoch Completed:  5 
Mean Squares Error of Epoch:  0.4595688872880123
Epoch Completed:  6 
Mean Squares Error of Epoch:  0.4565964606476218
Epoch Completed:  7 
Mean Squares Error of Epoch:  0.454582550361206
Epoch Completed:  8 
Mean Squares Error of Epoch:  0.4529248639451869
Epoch Completed:  9 
Mean Squares Error of Epoch:  0.4513846489814658
Epoch Completed:  10 
Mean Squares Error of Epoch:  0.449839911358233
Epoch Completed:  11 
Mean Squares Error of Epoch:  0.448210187087444
Epoch Completed:  12 
Mean Squares Error of Epoch:  0.44642934648783866
Epoch Completed:  13 
Mean Squares Error of Epoch:  0.4444307870059716
Epoch Completed:  14 
Mean Squares Error of Epoch:  0.44214066974090716
Epoch Completed: